In [7]:
import sys
from pathlib import Path

In [8]:
src_path = Path("../src").resolve()
sys.path.append(str(src_path))

In [9]:
from sqlmodel import Session, select
from api.db.session import engine
from api.events.models import EventModel
from timescaledb.hyperfunctions import time_bucket
from pprint import pprint

In [10]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.asc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)
    print("")
    print(str(query))
    results = session.exec(query).fetchall()
    pprint(results)

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT 10

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT :param_1
[EventModel(page='/test+', updated_at=datetime.datetime(2025, 4, 1, 4, 51, 53, 335165, tzinfo=datetime.timezone.utc), id=1, time=datetime.datetime(2025, 4, 1, 4, 51, 47, 609054, tzinfo=datetime.timezone.utc), description='inline test'),
 EventModel(page='/contact', updated_at=datetime.datetime(2025, 4, 1, 4, 54, 21, 558441, tzinfo=datetime.timezone.utc), id=2, time=datetime.datetime(2025, 4, 1, 4, 54, 21, 558420, tzinfo=datetime.timezone.utc), description=''),
 EventModel(page='/about', updated_at=datetime.datetime(2025, 4, 1, 4, 54, 21, 598212, tzinfo=datetime.timezone.utc), id=3, time=datetime.datetime(2025, 4, 1, 4, 54, 21, 598191, tzinfo=datetime.timezo

In [6]:
from sqlalchemy import func
from datetime import datetime, timedelta, timezone

with Session(engine) as session:
    bucket = time_bucket("1 day", EventModel.time)
    pages = ['/about', '/contact', '/pages', '/pricing']
    start = datetime.now(timezone.utc) - timedelta(hours=1)
    finish = datetime.now(timezone.utc) + timedelta(hours=1)
    query = (
        select(
            bucket,
            EventModel.page,
            func.count()
        )
        .where(
            EventModel.time > start,
            EventModel.time <= finish,
            EventModel.page.in_(pages)
        )
        .group_by(
            bucket,
            EventModel.page,
        )
        .order_by(
            bucket,
            EventModel.page,
        )
    )
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    # print(compiled_query)
    results = session.exec(query).fetchall()
    pprint(results)

    

[]
